In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 检查CUDA是否可用，并选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 添加：检查CUDA可用性并选择设备
print(f"Using device: {device}")  # 添加：打印当前使用的设备

# 定义神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(torch.max_pool2d(self.conv1(x), 2))
        x = torch.relu(torch.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# 加载MNIST数据集
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 初始化模型、优化器，并将模型移动到GPU
model = Net().to(device)  # 修改：将模型移动到GPU
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型函数
def train(model, train_loader, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)  # 修改：将数据移动到GPU
            optimizer.zero_grad()
            output = model(data)
            loss = torch.nn.functional.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

# 测试模型函数
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # 修改：将数据移动到GPU
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f'Accuracy on test set: {100 * correct / total}%')

# 开始训练和测试
try:
    model.load_state_dict(torch.load("./model/number.pth",weights_only=True))
    print("Model loaded from ./model/number.pth")
except FileNotFoundError:
    print("Model file not found, starting training...")
    train(model, train_loader, optimizer, epochs=5)
    torch.save(model.state_dict(), "./model/number.pth")
test(model, test_loader)


Using device: cuda
Model loaded from ./model/number.pth
Accuracy on test set: 98.81%


In [10]:
torch.save(model.state_dict(), "./model/number.pth")  # 保存模型参数到本地文件model.pth